In [1]:
%cd ..
import pandas as pd
import numpy as np
pd.set_option("display.max_columns", None)


/run/media/nazif/2F946E411BA61D49/thesis


In [2]:
df = pd.read_csv("results/2_positive_data.csv")

df.head()

,id,mrna_start,mrna_end,pred_energy,mirna_start,mirna_end,mirna_dot_bracket_5to3,mirna_sequence,mirna_accession,mre_region,enst,full_sequence_of_transcript,extended_mrna_start,extended_mrna_end,extended_mrna_sequence,clash_mrna_start,clash_mrna_end,mre_start,mre_end
0,0727A-1038930_1,22,45,-27.7,0,20,)))))))))))))))))))).,TGAGGTAGTAGGTTGTATAGTT,MIMAT0000062,TTATACAGAACCTACTGCCTCA,ENST00000340828,AGGGCAGGCGCGGCCCCTTCGGCTCCGAGCTGACCCTGATCAGGGC...,1760,1920,TTTAATATTTTTTTCTAGAAAACAGGTGACATTTGTATCTACGATA...,1790,1890,1813,1835
1,L1HS-1112536_1,11,30,-26.2,0,18,))))))))).)))))))).,TGAGGTAGTAGGTTGTATAGTT,MIMAT0000062,ACCCGTGCAACCAACTACCTCA,ENST00000343455,CGGAGGCGCGGCGCAGGCTGCTGCAGGCCCAGGTGAATGGAGTAAC...,3826,3958,TGCCAAGGAAATCAGCTAAATTACTACAAGCAGGAAATACCCGTGC...,3856,3928,3864,3886
2,L2HS-818542_2,3,24,-23.6,0,21,.)))))).))))))))))))).,TGAGGTAGTAGGTTGTATAGTT,MIMAT0000062,CCGCTATATGACCTGATGCCTT,ENST00000436639,GATTGCCAGGGCCGCCCTGTGCCCTCTGGCTCGGCGGTGGTGGGCG...,2354,2464,GCAGAACTCCTTTATGCTCTGAGAGCCATTACCCGCTATATGACCT...,2384,2434,2386,2408
3,L2HS-1161339_2,3,24,-23.8,0,21,)))))))...)))))))))))),TGAGGTAGTAGGTTGTATAGTT,MIMAT0000062,ATGACTATGCAACCATACCTTA,ENST00000282516,TCCGGTCGGCATTTTGTTCTGAGAGGGAGAGACGGAACGAGAGAGA...,6539,6653,AAATAAGACCCCAGCTCATGGTTAAACATGCAATGACTATGCAACC...,6569,6623,6571,6593
4,L2-407944_2,2,24,-24.0,0,21,))))))))))))))...)))).,TGAGGTAGTAGGTTGTATAGTT,MIMAT0000062,TACTGGGAAACCTGCTACTTCG,ENST00000340139,AGGGACCACCGGGAACAGACGGATCGGCAGGGCGGGGCGGAACGGT...,1133,1238,ACAGCAAAATTGAGTTCAACAACACAAAACAATACTGGGAAACCTG...,1163,1208,1165,1187


In [3]:
def find_mres_in_close_proximity_works(df):
    
    df["middle_of_mre"] = ((df["mre_start"] + df["mre_end"]) // 2)

    # Convert the 'middle_of_mre' column to a NumPy array
    middle_of_mre = df['middle_of_mre'].to_numpy()

    # Find the indices where the length of each group is greater than or equal to 2
    group_indices = df.groupby(["enst","mirna_accession"]).indices
    filtered_indices = [indices for indices in group_indices.values() if len(indices) >= 2]

    # Create a dictionary to store the results
    result = {}

    # Loop through the filtered indices and perform the computation
    for indices in filtered_indices:
        # Get the values corresponding to the indices
        values = middle_of_mre[indices]

        # Create a mask for the condition
        mask = np.logical_and(
            np.abs(values[:, None] - values) > (13+4),
            np.abs(values[:, None] - values) < (35+4)
        )

        # Get the indices where the condition is satisfied
        i, j = np.where(mask)

        # Add the pairs to the result dictionary
        for k in range(len(i)):
            # Create a key using the 'enst' and 'mirna_accession' values from the DataFrame
            key = (df.iloc[indices[i[k]]]['enst'], df.iloc[indices[i[k]]]['mirna_accession'])

            # If the key is not in the result dictionary, add it with an empty list as the value
            if key not in result:
                result[key] = []

            # Get the pair values
            a, b = values[i[k]], values[j[k]]

            # Check if the pair (a, b) or (b, a) is already in the list associated with the key
            if (a, b) not in result[key] and (b, a) not in result[key]:
                # Append the pair of values to the list associated with the key
                result[key].append((a, b))

    # Initialize a list to store the tuples
    data_list = []

    # Loop through the dictionary and append the values as tuples to the list
    for (enst, mirna), values in result.items():
        data_list.extend((enst, mirna, i) for i in values[0])

    df['in_close_proximity'] = df.apply(lambda row: 1 if (row['enst'], row['mirna_accession'], row['middle_of_mre']) in data_list else 0, axis=1)
    
    return df

df = find_mres_in_close_proximity_works(df)

df.head()

,id,mrna_start,mrna_end,pred_energy,mirna_start,mirna_end,mirna_dot_bracket_5to3,mirna_sequence,mirna_accession,mre_region,enst,full_sequence_of_transcript,extended_mrna_start,extended_mrna_end,extended_mrna_sequence,clash_mrna_start,clash_mrna_end,mre_start,mre_end,middle_of_mre,in_close_proximity
0,0727A-1038930_1,22,45,-27.7,0,20,)))))))))))))))))))).,TGAGGTAGTAGGTTGTATAGTT,MIMAT0000062,TTATACAGAACCTACTGCCTCA,ENST00000340828,AGGGCAGGCGCGGCCCCTTCGGCTCCGAGCTGACCCTGATCAGGGC...,1760,1920,TTTAATATTTTTTTCTAGAAAACAGGTGACATTTGTATCTACGATA...,1790,1890,1813,1835,1824,0
1,L1HS-1112536_1,11,30,-26.2,0,18,))))))))).)))))))).,TGAGGTAGTAGGTTGTATAGTT,MIMAT0000062,ACCCGTGCAACCAACTACCTCA,ENST00000343455,CGGAGGCGCGGCGCAGGCTGCTGCAGGCCCAGGTGAATGGAGTAAC...,3826,3958,TGCCAAGGAAATCAGCTAAATTACTACAAGCAGGAAATACCCGTGC...,3856,3928,3864,3886,3875,0
2,L2HS-818542_2,3,24,-23.6,0,21,.)))))).))))))))))))).,TGAGGTAGTAGGTTGTATAGTT,MIMAT0000062,CCGCTATATGACCTGATGCCTT,ENST00000436639,GATTGCCAGGGCCGCCCTGTGCCCTCTGGCTCGGCGGTGGTGGGCG...,2354,2464,GCAGAACTCCTTTATGCTCTGAGAGCCATTACCCGCTATATGACCT...,2384,2434,2386,2408,2397,0
3,L2HS-1161339_2,3,24,-23.8,0,21,)))))))...)))))))))))),TGAGGTAGTAGGTTGTATAGTT,MIMAT0000062,ATGACTATGCAACCATACCTTA,ENST00000282516,TCCGGTCGGCATTTTGTTCTGAGAGGGAGAGACGGAACGAGAGAGA...,6539,6653,AAATAAGACCCCAGCTCATGGTTAAACATGCAATGACTATGCAACC...,6569,6623,6571,6593,6582,0
4,L2-407944_2,2,24,-24.0,0,21,))))))))))))))...)))).,TGAGGTAGTAGGTTGTATAGTT,MIMAT0000062,TACTGGGAAACCTGCTACTTCG,ENST00000340139,AGGGACCACCGGGAACAGACGGATCGGCAGGGCGGGGCGGAACGGT...,1133,1238,ACAGCAAAATTGAGTTCAACAACACAAAACAATACTGGGAAACCTG...,1163,1208,1165,1187,1176,0


In [5]:
len(df[df.in_close_proximity == 1])

30

In [6]:
import numpy as np

def find_mres_in_close_proximity(df):
    # Calculate the middle of MRE
    df["middle_of_mre"] = ((df["mre_start"] + df["mre_end"]) // 2).astype(int)

    # Find the indices where the length of each group is greater than or equal to 2
    group_indices = df.groupby(["enst", "mirna_accession"]).indices
    filtered_indices = [indices for indices in group_indices.values() if len(indices) >= 2]

    # Create a dictionary to store the results
    result = {}

    # Loop through the filtered indices and perform the computation
    for indices in filtered_indices:
        # Get the values corresponding to the indices
        values = df.iloc[indices]["middle_of_mre"].values

        # Create a mask for the condition
        mask = np.logical_and(
            np.abs(values[:, None] - values) > (13 + 4),
            np.abs(values[:, None] - values) < (35 + 4)
        )

        # Get the indices where the condition is satisfied
        i, j = np.where(mask)

        # Add the pairs to the result dictionary
        for k in range(len(i)):
            # Create a key using the 'enst' and 'mirna_accession' values from the DataFrame
            key = (df.iloc[indices[i[k]]]['enst'], df.iloc[indices[i[k]]]['mirna_accession'])

            # If the key is not in the result dictionary, add it with an empty list as the value
            if key not in result:
                result[key] = []

            # Get the pair values
            a, b = values[i[k]], values[j[k]]

            # Check if the pair (a, b) or (b, a) is already in the list associated with the key
            if (a, b) not in result[key] and (b, a) not in result[key]:
                # Append the pair of values to the list associated with the key
                result[key].append((a, b))

    # Create a set of tuples for faster membership checking
    data_set = {(enst, mirna, i) for (enst, mirna), values in result.items() for i in values[0]}

    # Use vectorized operations to check membership and assign the result
    df['another_mre_in_close_proximity'] = np.where(
        df[['enst', 'mirna_accession', 'middle_of_mre']].apply(tuple, axis=1).isin(data_set),
        1,
        0
    )

    return df

df = find_mres_in_close_proximity(df)